# III. Filter data using Cordoni's algorithm

> Francisco Carrasco Varela - Pontificia Universidad Católica de Chile (PUC) - ffcarrasco@uc.cl ⭐

<mark>The following Jupyter Notebook is used to extract and work with Gaia EDR3 / Gaia DR3 data (and other data releases) </mark>

In [2]:
# Import all the libraries we will need

%matplotlib inline
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
import numpy as np
import os
from astropy.io import ascii
from astropy.table import vstack, Table

In [3]:
# Get data from Vasiliev (2019) file

###################################################
object_name = "NGC104"  # <--- EDITABLE
vasiliev_file = "Vasiliev_2019_Gaia_parameters.dat"
###################################################

class Error(Exception):
    """Base class for other exceptions"""
    pass


class ClusterNotFound(Error):
    """
    Custom error class to indicate the object provided by the user is not in the
    file provided by Vasiliev (2019) and Harris (2010)
    """
    pass


@dataclass(frozen=True, order=True, kw_only=True)
class VasilievData:
    """
    An object/struct used to store data loaded from Vasiliev (2019) file.
    """
    name: str #object name
    RA: float #object RA coordinates (J2000)
    DEC: float #object DEC coordinates (J2000)
    distance: float # Harris (2010) distance in kpc
    losv: float # line of sight velocity in km/s
    err_losv: float # error of line of sight velocity in km/s
    pm_RA : float # mean proper motion RA in mas / yr
    pm_DEC: float # mean proper motion DEC in mas / yr
    err_pm_RA : float # error mean proper motion RA in mas / yr
    err_pm_DEC : float # error mean proper motion DEC in mas / yr
    correlation: float # correlation coefficient (normalized non-diagonal element in the error covariance matrix)
    rh: float # angular scale radius (arcmin)
    N_stars: int # numbers of stars

        
@dataclass
class VasilievList:
    """
    Struct containing a list of VasilievData objects
    """
    data: list[VasilievData] = field(default_factory=list)
        

def get_GC_params(fname: str) -> VasilievList:
    """
    Reads data from file containing Vasiliev (2019) data. And returns a list with their characteristics
    in a dataclass.
    """
    data_type = np.dtype([("name", np.unicode_, 16), ("ra", float), ("dec", float), ("dist",float), ("hrv",float),
                         ("err_hrv", float), ("pmRA", float), ("pmDEC", float), ("err_pmRA", float), ("err_pmDEC", float),
                         ("corr", float), ("r_h",float), ("N_stars", int)])
    gc_names, gc_ra, gc_dec, dist, hrv, e_hrv, pmRA, pmDEC, e_pmRA, e_pmDEC, corr, gc_rh, n_stars = np.loadtxt(fname, 
                                                                                                               dtype=data_type, 
                                                                                                               comments='#',
                                                                                                               unpack=True)    
    data_vasiliev = VasilievList()
    
    for name_it, ra, dec, d, hr, ehr, pmra, pmdec, epmra, epmdec, c, rh_it, n in zip(gc_names, gc_ra, gc_dec, dist, hrv, e_hrv, pmRA, pmDEC, e_pmRA, e_pmDEC, corr, gc_rh, n_stars):
        data = VasilievData(name=name_it, RA=ra, DEC=dec, distance=d, losv=hr, err_losv=ehr, pm_RA=pmra, pm_DEC=pmdec,
                           err_pm_RA=epmra, err_pm_DEC=epmdec, correlation=c, rh=rh_it, N_stars=n)
        data_vasiliev.data += [data]
    
    return data_vasiliev

def get_selected_GC(obj_name: str, data_listed: VasilievList) -> (VasilievData, bool):
    """
    Returns a VasilievData type variable which name matches with that variable.name inside
    Vasiliev Data list.
    If the object (obj_name) is not found in data list (data_listed) then it will return an
    empty variable and it will raise an exception.
    """
    try:
        for item in data_listed.data:
            if item.name.upper() == object_name.upper():
                return item, True
        raise ClusterNotFound
        
    except ClusterNotFound:
        print(f"{obj_name!r} object not found in Globular Cluster list!")
        print("You will have to add coordinates manually in the next cell.")
        print()
        return VasilievData(name="", RA=0., DEC=0., distance=0., losv=0., err_losv=0.,
                            pm_RA=0., pm_DEC=0., err_pm_RA=0, err_pm_DEC=0., correlation=0,
                            rh=0., N_stars=0), False
           
data_list = get_GC_params(vasiliev_file)
obj, success = get_selected_GC(object_name, data_list)

if success:
    n_times = 60
    print("Object detected succesfully!")
    print("-"*n_times)
    print(f"Object name: {obj.name}")
    print(f"Mean Proper Motion RA (mas/yr): {obj.pm_RA} +- {obj.err_pm_RA}")
    print(f"Mean Proper Motion DEC (mas/yr): {obj.pm_DEC} +- {obj.err_pm_DEC}")
    print("-"*n_times)

Object detected succesfully!
------------------------------------------------------------
Object name: NGC104
Mean Proper Motion RA (mas/yr): 5.237 +- 0.039
Mean Proper Motion DEC (mas/yr): -2.524 +- 0.039
------------------------------------------------------------
